0. Scan glass slides at 20x
1. Create Excel sheet with patient information
2. Screen H&E images to select potentially healthy sections
3. Annotate the Region of Interest (ROI) (Optional)
4. Convert ROI XML to Image (Code)
5. Annotate 12 tissue compartments, apply DeepLab, and evaluate performance metrics
6. Apply HoVerNet
7. Rotate and crop tissue pieces and DLmask (Code)
8. Rotate HoVerNet segmented nuclei mask (Code)
9. Calculate tissue composition (Code)
10. Calculate epidermis thickness, waviness, roughness
11. Calculate dermis thickness, waviness
12. Calculate ECM anisotropy,Interstitial orientation, Thick gap ratio, Thick gap average
13. Calculate tissue morphology
14. ECM cell count and morphology + std using Manual Threshold method
15. Distance from ECM cell to tissue
16. All celltype morphology, orientation, and distance to tissue using HoVerNet
17. Further add features such as more detailed cell subtyping

In [1]:
import os
import glob
import pandas as pd
import numpy as np

#utilities
from roixml2png import roixml2png

In [2]:
NDPI = r'\\fatherserverdw\kyuex\clue images'
oneum = r'\\fatherserverdw\kyuex\clue images\1um'
imcrop = r'\\fatherserverdw\kyuex\clue images\1um\imcrop'
imcroproi = r'\\fatherserverdw\kyuex\clue images\1um\imcrop_roi'

dlmask = r'\\fatherserverdw\kyuex\clue images\1um\classification_v9_combined'
dlcrop = r'\\fatherserverdw\kyuex\clue images\1um\classification_v9_combined\dlcrop'
dlcroproi = r'\\fatherserverdw\kyuex\clue images\1um\classification_v9_combined\dlcrop_roi'

roixml = r'\\fatherserverdw\kyuex\clue images\annotations\roi'
roimask_small = r'\\fatherserverdw\kyuex\clue images\annotations\roi\labeledmask_20rsf'
tissue_area_small = r'\\fatherserverdw\kyuex\clue images\annotations\roi\TA_20rsf'

twelve_annot = r'\\fatherserverdw\kyuex\clue images\annotations\12class'

LUT = r"\\fatherserverdw\kyuex\imlist_all.xlsx"

In [3]:
#Step 4: ROI XML 2 Image
#Saves Image to where XML is in subfolders

roixml = r'\\fatherserverdw\kyuex\clue images\annotations\roi'
NDPI = r"\\fatherserverdw\kyuex\clue images"

LUT = pd.read_excel(LUT)
# xmlist = LUT['filename'][(LUT['student score']>1) & (LUT['Block or Slide?']=="Both")]
LUThealthy = LUT[(LUT['student score']>1)]
xmlist = [os.path.join(roixml,xmlpth.replace('ndpi','xml')) for xmlpth in LUThealthy['filename']]
roiinfo = [roixml2png(xmlpth,NDPI) for xmlpth in xmlist]
pd.DataFrame(np.array(roiinfo),columns=['fn','ROIA','TA','ratio']).to_csv(r"\\fatherserverdw\kyuex\ROITA_ratio.csv")

2022-06-07 17.10.29.xml
2022-06-07 17.15.33.xml
2022-06-07 16.05.45.xml
2022-06-09 20.12.48.xml
2022-06-09 20.18.08.xml
2022-06-09 20.52.52.xml
2022-06-10 20.40.21.xml
2022-06-07 17.05.33.xml
2022-06-09 21.14.27.xml
2022-06-09 22.06.26.xml
2022-06-13 15.50.57.xml
2022-06-09 22.00.47.xml
2022-06-10 18.30.09.xml
2022-06-30 23.12.42.xml
2022-06-09 21.42.40.xml
2022-06-09 18.31.45.xml
2022-06-09 18.34.58.xml
2022-06-09 18.52.30.xml
2022-06-09 19.55.38.xml
2022-06-09 20.02.00.xml
2022-06-09 21.34.03.xml
2022-06-09 22.12.56.xml
2022-06-09 21.38.17.xml
2022-07-01 00.08.09.xml
2022-06-10 20.25.05.xml
2022-06-30 23.47.44.xml
2022-06-10 20.21.32.xml
2022-06-30 23.35.09.xml
2022-07-01 00.02.03.xml
2022-06-30 20.21.27.xml
2022-06-10 18.00.26.xml
2022-06-07 17.43.53.xml
2022-06-07 17.48.30.xml
2022-07-07 22.11.00.xml
2022-06-09 16.46.04.xml
2022-06-09 16.53.12.xml
2022-06-09 16.56.40.xml
2022-07-07 22.16.04.xml
2022-06-09 12.15.00.xml
2022-06-09 18.02.27.xml
2022-07-07 12.04.08.xml
2022-06-09 19.37

FileNotFoundError: [Errno 2] No such file or directory: '\\\\fatherserverdw\\kyuex\\clue images\\annotations\\roi\\2022-06-14 13.34.41.xml'